# Análise Estatística — Student Performance

**Arquivo:** `Student_performance_data.csv`

**Objetivo:** Aplicar conceitos de inferência estatística (testes de hipótese e intervalos de confiança via bootstrap ou métodos analíticos) usando Python. Este notebook contém: exploração de dados, formulação de hipóteses, escolha e aplicação de testes, bootstrap para ICs e visualizações.

**Data de geração:** 2025-10-16 15:28:35 UTC

## Estrutura do notebook

1. Carregar dados
2. Análise exploratória (tipos, valores ausentes, estatísticas)
3. Formulação de hipóteses
4. Testes de hipótese com verificação de pressupostos
5. Intervalos de confiança por bootstrap
6. Visualizações
7. Conclusões

---

In [ ]:
# Imports e carregamento de dados
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Caminho do arquivo (presente no ambiente)
path = '/mnt/data/Student_performance_data.csv'
df = pd.read_csv(path)
print('Dados carregados. Shape:', df.shape)
df.head()


In [ ]:
# Exploração inicial
display(df.dtypes.to_frame(name='dtype').assign(n_unique=df.nunique(), n_missing=df.isna().sum()))
print('\nResumo estatístico (numérico):\n')
display(df.describe().T)
print('\nResumo (todas colunas):')
display(df.describe(include='all').T)

print('Colunas:', df.columns.tolist())
possible_score_cols = [c for c in df.columns if any(s in c.lower() for s in ['gpa','grade','score','final','mark'])]
print('Colunas relacionadas a notas encontradas:', possible_score_cols)

if 'GPA' in df.columns:
    df['avg_score'] = df['GPA']
elif possible_score_cols:
    df['avg_score'] = df[possible_score_cols].mean(axis=1)

if 'Gender' in df.columns:
    print('\nContagem por Gender:\n', df['Gender'].value_counts())

display(df.head())


## Formulação das Hipóteses

As hipóteses criadas neste notebook:

- **H1 (diferença entre gêneros):**
  - H0: Não há diferença significativa entre as médias de **GPA** dos dois grupos de `Gender`.
  - H1: Há diferença significativa entre as médias de **GPA** dos dois grupos.

- **H2 (correlação):**
  - H0: Não há correlação entre **StudyTimeWeekly** e **GPA**.
  - H1: Existe correlação significativa entre **StudyTimeWeekly** e **GPA**.

Justificativa: `Gender` possui 2 grupos independentes → teste de duas amostras independentes (t-test ou não-paramétrico). Para correlação entre duas contínuas, Pearson (se normal) ou Spearman (se não-normal).

In [ ]:
# Testes de hipótese e bootstrap para ICs
alpha = 0.05
results = {}

if 'Gender' in df.columns:
    unique_g = sorted(df['Gender'].unique())
    if len(unique_g) >= 2:
        gender_map = {unique_g[0]:'Group_0', unique_g[1]:'Group_1'}
        df['gender_label'] = df['Gender'].map(gender_map)

if 'gender_label' in df.columns and 'GPA' in df.columns:
    df_h1 = df[['gender_label','GPA']].dropna()
    groups = df_h1['gender_label'].unique()
    g1, g2 = groups[:2]
    grp1 = df_h1[df_h1['gender_label']==g1]['GPA']
    grp2 = df_h1[df_h1['gender_label']==g2]['GPA']

    sh1 = stats.shapiro(grp1) if len(grp1)<=5000 else None
    sh2 = stats.shapiro(grp2) if len(grp2)<=5000 else None
    levene = stats.levene(grp1, grp2)
    use_t = True
    if sh1 is not None and sh2 is not None:
        if sh1.pvalue < 0.05 or sh2.pvalue < 0.05:
            use_t = False

    if use_t:
        tstat, tp = stats.ttest_ind(grp1, grp2, equal_var=(levene.pvalue>0.05), nan_policy='omit')
        test_used = f"t-test independent (equal_var={levene.pvalue>0.05})"
        stat, pval = tstat, tp
    else:
        u, up = stats.mannwhitneyu(grp1, grp2, alternative='two-sided')
        test_used = 'Mann-Whitney U (two-sided)'
        stat, pval = u, up

    rng = np.random.default_rng(2025)
    n_boot = 5000
    boots = []
    for _ in range(n_boot):
        s1 = rng.choice(grp1, size=len(grp1), replace=True)
        s2 = rng.choice(grp2, size=len(grp2), replace=True)
        boots.append(s1.mean() - s2.mean())
    boots = np.array(boots)
    ci_low, ci_high = np.percentile(boots, [2.5,97.5])

    results['H1'] = {
        'groups': (g1, g2),
        'n': (len(grp1), len(grp2)),
        'shapiro_p': (sh1.pvalue if sh1 is not None else None, sh2.pvalue if sh2 is not None else None),
        'levene_p': levene.pvalue,
        'test_used': test_used,
        'statistic': float(stat),
        'pvalue': float(pval),
        'mean_diff': float(grp1.mean() - grp2.mean()),
        'grp1_mean': float(grp1.mean()),
        'grp2_mean': float(grp2.mean()),
        'bootstrap_diff_mean_95CI': (float(ci_low), float(ci_high))
    }
else:
    results['H1'] = {'error':'colunas gender_label e/ou GPA ausentes'}

# H2: correlação
if 'StudyTimeWeekly' in df.columns and 'GPA' in df.columns:
    df_h2 = df[['StudyTimeWeekly','GPA']].dropna()
    x = df_h2['StudyTimeWeekly']
    y = df_h2['GPA']
    shx = stats.shapiro(x) if len(x)<=5000 else None
    shy = stats.shapiro(y) if len(y)<=5000 else None
    use_pearson = True
    if shx is not None and shy is not None:
        if shx.pvalue < 0.05 or shy.pvalue < 0.05:
            use_pearson = False
    if use_pearson:
        r, p_corr = stats.pearsonr(x, y)
        corr_test = 'Pearson r'
    else:
        r, p_corr = stats.spearmanr(x, y)
        corr_test = 'Spearman rho'
    rng = np.random.default_rng(7)
    n_boot = 5000
    boots_r = []
    n = len(x)
    for _ in range(n_boot):
        idx = rng.integers(0,n,n)
        xr = x.iloc[idx].to_numpy()
        yr = y.iloc[idx].to_numpy()
        try:
            if use_pearson:
                rr, _ = stats.pearsonr(xr, yr)
            else:
                rr, _ = stats.spearmanr(xr, yr)
            boots_r.append(rr)
        except Exception:
            boots_r.append(np.nan)
    boots_r = np.array(boots_r)
    boots_r = boots_r[~np.isnan(boots_r)]
    ci_r = np.percentile(boots_r, [2.5,97.5])
    results['H2'] = {
        'n': len(x),
        'test_used': corr_test,
        'statistic_r': float(r),
        'pvalue': float(p_corr),
        'bootstrap_r_95CI': (float(ci_r[0]), float(ci_r[1])),
        'shapiro_p': (shx.pvalue if shx is not None else None, shy.pvalue if shy is not None else None)
    }
else:
    results['H2'] = {'error':'colunas StudyTimeWeekly e/ou GPA ausentes'}

import pprint
pprint.pprint(results)

summary_rows = []
for k,v in results.items():
    row = {'hypothesis':k}
    row.update(v)
    summary_rows.append(row)
summary_df = pd.DataFrame(summary_rows)
summary_df.to_csv('/mnt/data/summary_hypothesis_tests_notebook.csv', index=False)
print('\nResumo salvo em /mnt/data/summary_hypothesis_tests_notebook.csv')


In [ ]:
# Visualizações
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0, 'figure.figsize': (8,4)})

if 'gender_label' in df.columns and 'GPA' in df.columns:
    plt.figure()
    for g in df['gender_label'].unique():
        subset = df[df['gender_label']==g]['GPA']
        plt.hist(subset, bins=20, alpha=0.4, label=str(g))
    plt.xlabel('GPA'); plt.ylabel('count'); plt.title('Histogram of GPA by gender'); plt.legend(); plt.show()

if 'gender_label' in df.columns and 'GPA' in df.columns:
    plt.figure()
    groups = [df[df['gender_label']==g]['GPA'] for g in df['gender_label'].unique()]
    plt.boxplot(groups, labels=df['gender_label'].unique()); plt.ylabel('GPA'); plt.title('Boxplot of GPA by gender'); plt.show()

if 'gender_label' in df.columns and 'GPA' in df.columns:
    means = []
    errs = []
    labels = []
    rng = np.random.default_rng(0)
    for g in df['gender_label'].unique():
        grp = df[df['gender_label']==g]['GPA'].dropna()
        labels.append(g)
        means.append(grp.mean())
        boots = [rng.choice(grp, size=len(grp), replace=True).mean() for _ in range(2000)]
        ci = np.percentile(boots, [2.5,97.5])
        errs.append([means[-1]-ci[0], ci[1]-means[-1]])
    means = np.array(means); errs = np.array(errs).T
    x = np.arange(len(labels))
    plt.figure(); plt.errorbar(x, means, yerr=errs, fmt='o', capsize=5); plt.xticks(x, labels); plt.ylabel('GPA mean'); plt.title('Mean ± 95% CI (bootstrap) by gender'); plt.show()

if 'StudyTimeWeekly' in df.columns and 'GPA' in df.columns:
    df_sc = df[['StudyTimeWeekly','GPA']].dropna()
    x = df_sc['StudyTimeWeekly'].to_numpy(); y = df_sc['GPA'].to_numpy()
    plt.figure(); plt.scatter(x, y, alpha=0.6)
    try:
        m,b = np.polyfit(x,y,1)
        xp = np.linspace(x.min(), x.max(), 100)
        plt.plot(xp, m*xp + b)
    except Exception:
        pass
    plt.xlabel('StudyTimeWeekly'); plt.ylabel('GPA'); plt.title('StudyTimeWeekly vs GPA'); plt.show()


## Conclusões

- **H1 (GPA por gênero):** teste não encontrou evidência de diferença entre os dois grupos (p > 0.05). O IC bootstrap para a diferença de médias contém 0.

- **H2 (StudyTimeWeekly × GPA):** correlação positiva e estatisticamente significativa (Spearman rho). IC bootstrap para rho não inclui 0.

Essas conclusões são descritivas e baseadas nos dados fornecidos. Para análise causal ou preditiva, recomenda-se modelagem multivariada e considerar potenciais variáveis confundidoras.

---

### Arquivos gerados neste diretório `/mnt/data`:
- `summary_hypothesis_tests_notebook.csv` — resumo dos testes e estatísticas.
- `Student_performance_analysis.ipynb` — este notebook.

Se quiser que eu gere também um PDF do notebook ou rodar análises adicionais (regressão múltipla, análise por subgrupos, etc.), eu posso fazer já.